In [1]:
import cv2
import requests
import numpy as np
from ultralytics import YOLO
import time

# Initialize YOLO model
model = YOLO('yolov8s.pt')

# URLs of the images
image_urls = [
    'http://192.168.170.5/capture',
    'http://192.168.170.124/capture'
]

dic = {
    'http://192.168.170.5/capture' : 1,
    'http://192.168.170.124/capture' : 2
}

In [2]:
# Function to download and decode an image from a URL
def download_image(url, retries=10, delay=1):  
    for attempt in range(retries):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                image_array = np.frombuffer(response.content, dtype=np.uint8)
                image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
                return image
        except requests.RequestException as e:
            pass
        
        time.sleep(delay)
    
    raise Exception(f"Failed to download image after {retries} attempts.")

# Function to detect vehicles in an image
def detect_vehicles(image):
    results = model(image)
    detections = results[0].boxes
    vehicle_classes = [2, 3, 5, 7]  # Car, motorcycle, bus, truck
    vehicle_detections = [d for d in detections if d.cls in vehicle_classes]
    return vehicle_detections

# Initialize voting tracking
votes = {}

In [4]:
# Function to update ThingSpeak channel
def update_thingspeak(value):
    url = f'https://api.thingspeak.com/update?api_key=Q8FYKER3B2F0PZ12&field1={value}'
    response = requests.get(url)
    if response.status_code == 200:
        print(f"ThingSpeak update successful: {value}")
    else:
        print(f"ThingSpeak update failed. Status code: {response.status_code}")

# Main loop to process images every 20 seconds
while True:
    images = [download_image(url) for url in image_urls]
    vehicle_counts = [len(detect_vehicles(image)) for image in images]

    max_vehicles = max(vehicle_counts)
    max_index = vehicle_counts.index(max_vehicles)
    
    # Add to history and voting
    chosen_url = image_urls[max_index]
    print(chosen_url)
    if chosen_url not in votes.keys():
        votes[chosen_url] =1
    else : 
        votes[chosen_url] += 1
    
    # Check for elimination condition
    eliminated_url = None
    for url, count in votes.items():
        if count >= 3:
            eliminated_url = url
            votes={}
            break
    green = dic[chosen_url]
    if eliminated_url :
        if eliminated_url == chosen_url :
            if  eliminated_url != 'http://192.168.170.5/capture' :
                green = 2
            else :
                green = 1
            
        print(f"Image from {eliminated_url} has been eliminated after 3 votes.")
    
    # Check if the vehicle count exceeds the threshold to trigger the red LED
    update_thingspeak(green)

    # Wait for 20 seconds before next iteration
    time.sleep(15)


0: 480x640 (no detections), 481.1ms
Speed: 5.6ms preprocess, 481.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 492.7ms
Speed: 18.4ms preprocess, 492.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
http://192.168.170.5/capture
ThingSpeak update successful: 1

0: 480x640 1 person, 435.2ms
Speed: 3.0ms preprocess, 435.2ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 458.1ms
Speed: 6.2ms preprocess, 458.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
http://192.168.170.5/capture
ThingSpeak update successful: 1

0: 480x640 1 cell phone, 451.6ms
Speed: 3.9ms preprocess, 451.6ms inference, 4.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 436.7ms
Speed: 3.1ms preprocess, 436.7ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
http://192.168.170.5/capture
Image from http://192.168.170.5/capture has been

Exception: Failed to download image after 10 attempts.